In [1]:
import time as t
import datetime as dt
import pandas as pd
import os
import sys
sys.path.insert(1, '../')
from utils.common import *

In [4]:
with SQLiteDB('../dbs/calls.db') as conn:
    query = "SELECT * FROM calls "  #WHERE launched is NULL
    df = pd.read_sql_query(query, conn)
    df = df.loc[dt.datetime.now() > pd.to_datetime(df['expected_launch_time_ts']) + dt.timedelta(minutes=5)]
df

,id,date,address,expected_launch_time,expected_launch_time_ts,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num,launched,launch_time,buy,buy_time,buy_price
0,212902,2024-04-16 09:44:54,AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy,27 seconds ago,2024-04-16 09:44:27,0,0,0,0,2,1270,1780,NaN,None,1.0,2024-04-28 18:31:06,0.000016
1,215618,2024-04-18 14:35:57,CfQSSRk6Md2cnZCzkJu2QjAVua5FzGMCua7CrhQGRURK,In 1 minute,2024-04-18 14:36:57,0,0,0,2,0,9050,18110,1.0,2024-04-18 14:37:00,NaN,None,NaN
2,215619,2024-04-18 14:37:11,GKMRt3xZznGK4wxoBw3SZfMTwVR9LLFTJxbqMCUvMabg,11 seconds ago,2024-04-18 14:37:00,1,0,0,0,2,1390,2500,1.0,2024-04-18 14:37:00,NaN,None,NaN
3,215620,2024-04-18 14:37:58,66Yp2kKFLsEjszAMVttwHFDZRZRyGt9kbKoXnuv99Dx3,36 seconds ago,2024-04-18 14:37:22,1,0,0,0,4,1730,1390,1.0,2024-04-18 14:42:00,NaN,None,NaN
4,215621,2024-04-18 14:38:20,Ecz4mak5jScXrwZkMGJB3mqqpeuJPWPTddJW6Rgmt9Za,27 seconds ago,2024-04-18 14:37:53,0,0,0,0,2,1110,1110,1.0,2024-04-18 14:38:00,NaN,None,NaN
5,215622,2024-04-18 14:39:15,BdSSmAmWAE8F2yhjAPW91Boa4fQfu98EKt3xxgLKQVxn,36 seconds ago,2024-04-18 14:38:39,1,1,0,0,2,40230,80460,1.0,2024-04-18 14:39:00,NaN,None,NaN
6,218927,2024-04-21 01:14:29,HbzH8fFYQRLjHgDK7G7qXYzWdV4fnwTSdb71zAZvJyDS,14 seconds ago,2024-04-21 01:14:15,0,0,0,2,0,1650,3290,1.0,2024-04-21 01:14:00+00:00,NaN,None,NaN
7,218928,2024-04-21 01:14:51,6xPz9dztCA6TdMsPZ6AHAZ6NEdGMjrouLb3fov8vbBJF,476017 hours and 14 minutes ago,2024-04-21 01:14:51,0,0,0,0,1,770,1500,1.0,2024-04-21 01:14:00,NaN,None,NaN
8,218929,2024-04-21 01:16:28,6XejiuxHwr31q7iZr4bYWzuNzzKoQWh4X4rrYP1qkt6t,13 seconds ago,2024-04-21 01:16:15,1,1,0,0,3,1490,2990,1.0,2024-04-21 01:16:00,NaN,None,NaN
9,218930,2024-04-21 01:17:25,7VEqMtsR2bRFU3g5cMTHuCYHdR6x1cwoVfoQ1uenJVqs,476017 hours and 17 minutes ago,2024-04-21 01:17:25,0,0,0,2,0,57080,23620,1.0,2024-04-21 01:17:00,NaN,None,NaN
